# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
#By this query we will create new keyspace sparkifydb
session.execute("""CREATE KEYSPACE IF NOT EXISTS sparkifydb
        WITH REPLICATION = 
        {'class' : 'SimpleStrategy', 'replication_factor': 1}""")

#### Set Keyspace

In [23]:
# In This We set newly created namespace
session.set_keyspace("sparkifydb")

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'








# For Query 1:
#### To answer this question we will need to obtain (select) the artist name, song name, and song length from out table, and we will need to filter by sessionId and itemInSession. 
### In CQL our query looks like:
#### SELECT artist_name, song_title, song_length FROM SongSessionDetail WHERE sessionId = 338 AND itemInSession = 4

#### We create table according to query1 that is SongSessionDetail.
#### It will have primary key sessionid as partitiion key and itemInSession as Clustering Key. 



In [9]:

drop_statement_for_table1="drop table if exists SongSessionDetail"
create_statement_for_query1="""CREATE TABLE IF NOT EXISTS SongSessionDetail
(session_id int,item_in_session int,artist_name text,song_title text,song_length float,
PRIMARY KEY(session_id,item_in_session))"""
select_query1="select artist_name,song_title,song_length from SongSessionDetail where session_id= %s and item_in_session=%s"

session.execute(create_statement_for_query1)
                    

In [10]:
# here we write CSV to Apache Cassandra Database.
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO SongSessionDetail (session_id,item_in_session,artist_name,song_title,song_length)"
        query = query + " VALUES (%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
## SELECT statement to verify the data was entered into the table
rows=session.execute(select_query1,(338,4))

for row in rows:
    print (row)

Row(artist_name='Faithless', song_title='Music Matters (Mark Knight Dub)', song_length=495.30731201171875)


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

#  For Query2: 

#### We have to select  artist_name, song (sorted by itemInSession) and user first and last name) for userid = 10, sessionid = 182

### In CQL our query looks like:

#### SELECT itemInSession, artist_name, song_title, firstname, lastname FROM SongUserSessionDetail WHERE user_id = 10 AND session_id = 182

#### We will name our table SongUserSessionDetail
#### Our primary key will consist of composite partition key (user_id, session_id). 
#### The reason for this is that if we only use userId as partition key, the sessionid which belongs to the same user will be put into different nodes, which will have the performance issue when the volume of data is large.
#### Our clustering key will be itemInSession so that our results are order by it.
#### The columns of our table will be: user_id, session_id, item_in_session, artist_name, song_title and firstname and lastname.


In [21]:
## Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)
drop_statement_for_table2="drop table if exists SongUserSessionDetail"
create_statement_for_query2="""CREATE TABLE IF NOT EXISTS SongUserSessionDetail
(user_id int,session_id int,item_in_session int,artist_name text,song_title text,firstname text,lastname text,
PRIMARY KEY((user_id,session_id),item_in_session))"""
select_query2="select item_in_session,artist_name,song_title,firstname,lastname from SongUserSessionDetail where user_id=%s and session_id= %s"


session.execute(create_statement_for_query2)


# Insertion for Table 2
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO SongUserSessionDetail(user_id, session_id, item_in_session, artist_name, \
                                            song_title, firstname, lastname)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## We will assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

        
                    

In [22]:
#select query on table 2
rows=session.execute(select_query2,(10,182))

for row in rows:
    print (row)

Row(item_in_session=0, artist_name='Down To The Bone', song_title="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(item_in_session=1, artist_name='Three Drives', song_title='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(item_in_session=2, artist_name='Sebastien Tellier', song_title='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(item_in_session=3, artist_name='Lonnie Gordon', song_title='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')



# For Query3:
#### We have to provide  user name (first and last)  who listened to the song 'All Hands Against His Own'
To answer this question we have to select the user first name and lastname from out table, and we will need to filter by song name. we will add the column user_id to uniquely identify users.

### CQL Query:

#### SELECT firstname, lastname FROM SongUserDetail WHERE song_title = 'All Hands Against His Own'

#### We will name our table SongUserDetail

#### Our primary key will consist of partition key song_title, and clustering key userId. This uniquely identifies our rows.
#### The columns of our table will be: song_title, firstname, lastname and user_id.

In [19]:
## Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
drop_statement_for_table3="drop table if exists SongUserDetail"
create_statement_for_query3="""CREATE TABLE IF NOT EXISTS SongUserDetail
(song_title text,user_id int,firstname text,lastname text,
PRIMARY KEY(song_title,user_id))"""
select_query3="select firstname,lastname from SongUserDetail where song_title=%s"

session.execute(create_statement_for_query3)


In [20]:
# Insert in for SongUserDetail
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO SongUserDetail(song_title, user_id, firstname, lastname)"
        query = query + "VALUES (%s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [21]:
#Select for Query 3
rows=session.execute(select_query3,('All Hands Against His Own',))

for row in rows:
    print (row)

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

## We will Drop the table before closing out the sessions

In [ ]:
session.execute(drop_statement_for_table1)
session.execute(drop_statement_for_table2)
session.execute(drop_statement_for_table3)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()